In [2]:
import pandas as pd
import numpy as np

In [22]:
v = 2

### 1. solar power generation

In [23]:
excel_file = pd.ExcelFile(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/realne_report_solar_30_v0000{v}_20220301_20230228.xlsx")

data = pd.DataFrame()

# Loop through all sheets in the Excel file
for sheet_name in excel_file.sheet_names:
    # Load the sheet as a DataFrame
    df = excel_file.parse(sheet_name)
    
    # Add a new column to the DataFrame
    df["new_column_name"] = sheet_name
    columns = list(df.loc[4].values[:-1])
    columns.append('date')
    
    df.drop([0,1,2,3,4],inplace=True)
    df.reset_index(drop=True,inplace=True)
    df.columns = columns
    df.date = pd.to_datetime(df.date + ' ' + df.時刻, format="%Y-%m-%d %H:%M")
    df.drop('時刻', axis=1, inplace=True)
    data = pd.concat([data, df])

## 2. power consumption

In [24]:
excel_file1 = pd.ExcelFile(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/realne_report_surplus30p_v0000{v}_20220301_20221231.xlsx")

surplus_data = pd.DataFrame()

# Loop through all sheets in the Excel file
for sheet_name in excel_file1.sheet_names:
    # Load the sheet as a DataFrame
    df = excel_file1.parse(sheet_name)
    
    # Add a new column to the DataFrame
    df["new_column_name"] = sheet_name
    columns = list(df.loc[4].values[:-1])
    columns.append('date')
    
    df.drop([0,1,2,3,4],inplace=True)
    df.reset_index(drop=True,inplace=True)
    df.columns = columns
    df.date = pd.to_datetime(df.date + ' ' + df.時刻, format="%Y-%m-%d %H:%M")
    df.drop('時刻', axis=1, inplace=True)
    surplus_data = pd.concat([surplus_data, df])

In [25]:
excel_file2 = pd.ExcelFile(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/realne_report_surplus30p_v0000{v}_20230101_20230228.xlsx")

# Loop through all sheets in the Excel file
for sheet_name in excel_file2.sheet_names:
    # Load the sheet as a DataFrame
    df = excel_file2.parse(sheet_name)
    
    # Add a new column to the DataFrame
    df["new_column_name"] = sheet_name
    columns = list(df.loc[4].values[:-1])
    columns.append('date')
    
    df.drop([0,1,2,3,4],inplace=True)
    df.reset_index(drop=True,inplace=True)
    df.columns = columns
    df.date = pd.to_datetime(df.date + ' ' + df.時刻, format="%Y-%m-%d %H:%M")
    df.drop('時刻', axis=1, inplace=True)
    surplus_data = pd.concat([surplus_data, df])

## 3. cloud 

In [26]:
v1_cloud = pd.read_csv(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/cloud_v0000{v}.csv")
v1_cloud.index = pd.to_datetime(v1_cloud.target_date)
v1_cloud.drop(['execution_date','target_date'],axis=1,inplace=True)

## 4. solar

In [27]:
v1_solar = pd.read_csv(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/solar_v0000{v}.csv")
v1_solar.index = pd.to_datetime(v1_solar.target_date)
v1_solar.drop(['execution_date','target_date'],axis=1,inplace=True)

## 5. weather

In [28]:
v1_weather = pd.read_csv(f"../raw data/2023_devday_data-20230320T123236Z-001/2023_devday_data/v{v}/weather_forecast_v0000{v}.csv")
v1_weather.index = pd.to_datetime(v1_weather.target_date)
v1_weather.drop(['execution_date','target_date'],axis=1,inplace=True)

## 6. merge data

In [29]:
data.sort_values(by='date',inplace=True)
surplus_data.sort_values(by='date',inplace=True)
final_v1 = pd.merge(data, surplus_data, how='outer', on='date')
final_v1.index = final_v1.date
final_v1 = final_v1.tz_localize('Japan')

v1_cloud = v1_cloud.tz_convert('Japan')
v1_solar = v1_solar.tz_convert('Japan')
v1_weather = v1_weather.tz_convert('Japan')

final_v1 = pd.merge(final_v1, v1_cloud, how='outer', left_index=True, right_index=True)
final_v1 = pd.merge(final_v1, v1_solar, how='outer', left_index=True, right_index=True)
final_v1 = pd.merge(final_v1, v1_weather, how='outer', left_index=True, right_index=True)

## 7. save data

In [30]:
final_v1.to_csv(f'v{v}.csv')